In [1]:
from landlab import RasterModelGrid
from landlab.plot.imshow import imshow_node_grid
from landlab.components import PresFlowNetwork
#from pressurized_flow import PresFlowNetwork
#import landlab.components

In [2]:
mg = RasterModelGrid((40,50),100)
junc_elev = mg.add_zeros('node', 'junction__elevation')
d_h = mg.add_ones('link', 'hydraulic__diameter')
h = mg.add_zeros('node', 'hydraulic__head')
f = mg.add_ones('link', 'friction__factor')
#h = np.random.rand(mg.number_of_nodes)
Q = mg.add_zeros('link', 'conduit__discharge')
#net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)
#set boundary node head

h[mg.nodes_at_left_edge] = 10.
h[mg.nodes_at_right_edge] = 0.
h[mg.nodes_at_top_edge] = 0.
h[mg.nodes_at_bottom_edge] = 0.
mg.set_closed_boundaries_at_grid_edges(False,True,False,True)
Q[mg.active_links] = 1.
n_core = mg.number_of_core_nodes
links = mg.links_at_node
print "Number of links = ", mg.number_of_links
print "Number of nodes = ", mg.number_of_nodes
print "Number of active links = ", mg.number_of_active_links
print "Number of core nodes = ", mg.number_of_core_nodes


Number of links =  3910
Number of nodes =  2000
Number of active links =  3638
Number of core nodes =  1824


In [3]:

pfn = PresFlowNetwork(mg)


In [4]:
pfn.run_one_step()

Number of iterations = 1 tolerance = 1.46420918242
Number of iterations = 2 tolerance = 0.549172446635
Number of iterations = 3 tolerance = 0.194243666714
Number of iterations = 4 tolerance = 0.0486788065998
Number of iterations = 5 tolerance = 0.0136260152458
Number of iterations = 6 tolerance = 0.00449772672093
Number of iterations = 7 tolerance = 0.00160853948538
Number of iterations = 8 tolerance = 0.000541710750203


In [5]:

%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


/home/mcoving/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [6]:
figure()
imshow_node_grid(mg, h)